# Importing Modules and Loading Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [3]:
tweets_df = pd.read_csv('Datasets/tweets_sns.csv')
tweets_df.head()

,user_name,user_location,text,likes,date
0,klbe1997,الوادي الجديد,شعري لونه orange وانتوا؟!,0,2022-08-28
1,ws_23s,الوادي الجديد,انا بقول ان كفاية قوي صفقة اللاعب السنغالي ندا...,2,2022-08-28
2,thvsimpact,الوادي الجديد,جماعة الموضوع بقى بضان و ماسخ و سمج لو بيواعد ...,4,2022-08-28
3,dyna86889772,الوادي الجديد,شعري لونه orange وانتوا؟!,0,2022-08-28
4,ws_23s,الوادي الجديد,"""اللهمَّ غيرني لما تريد أنت، واختر لي ما تشاء ...",1,2022-08-28


In [8]:
tweets_df[tweets_df['text'].lower().str.contains("vodafone|فودافون|etisalat|اتصالات|orange|اورانج|موبينيل")]

AttributeError: 'Series' object has no attribute 'lower'